Step 1: Implementation of Gooseneck Barnacle Optimization (GBO)

In [ ]:
import numpy as np
from math import gamma, sin, pi

# Objective function (Example: Sphere function)
def objective_function(x):
    return np.sum(x**2)

# Initialization of GBO
def initialize_population(pop_size, dimensions, lb, ub):
    population = lb + (ub - lb) * np.random.rand(pop_size, dimensions)
    return population

# Levy Flight function
def levy_flight(beta, dimensions):
    sigma = (gamma(1 + beta) * sin(pi * beta / 2) /
             (gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
    u = np.random.randn(dimensions) * sigma
    v = np.random.randn(dimensions)
    step = u / np.abs(v) ** (1 / beta)
    return step

# Gooseneck Barnacle Optimization (GBO) Algorithm
def GBO(objective, pop_size, dimensions, lb, ub, max_iter):
    # Initialize Gooseneck Barnacle positions
    population = initialize_population(pop_size, dimensions, lb, ub)
    fitness = np.array([objective(ind) for ind in population])
    global_best = population[np.argmin(fitness)]
    global_best_fitness = np.min(fitness)

    for iter in range(max_iter):
        # Update significant wave height (Hs)
        Hs = 1.5 - iter * ((1.5 - 0.2) / max_iter)

        for i in range(pop_size):
            # Calculate distance (logarithmic spiral)
            Di = population[i] - global_best
            b = 0.5  # constant shape of spiral
            t = np.random.uniform(-1, 1)
            spiral = Di * np.exp(b * t) * np.cos(2 * np.pi * t) + global_best

            # Movement vectors calculation (Eq. 6 and Eq. 7 from your paper)
            WD = np.random.uniform(0, 2 * np.pi)
            Tdim = global_best
            delta_X = WD + Tdim + Hs * (population[i] - spiral)
            new_position = population[i] + delta_X

            # Boundary checking
            new_position = np.clip(new_position, lb, ub)

            # Evaluate fitness of new position
            new_fitness = objective(new_position)

            # Update positions
            if new_fitness < fitness[i]:
                population[i] = new_position
                fitness[i] = new_fitness

        # Levy flight condition (Eq. 8 from your paper)
        if np.random.rand() < 0.5:
            for i in range(pop_size):
                step = levy_flight(1.5, dimensions)
                levy_position = population[i] + step * population[i]
                levy_position = np.clip(levy_position, lb, ub)
                levy_fitness = objective(levy_position)

                if levy_fitness < fitness[i]:
                    population[i] = levy_position
                    fitness[i] = levy_fitness

        # Update global best
        current_best = np.min(fitness)
        if current_best < global_best_fitness:
            global_best_fitness = current_best
            global_best = population[np.argmin(fitness)]

        # Optional: Display iteration progress
        print(f'Iteration {iter+1}/{max_iter}, Best Fitness: {global_best_fitness}')

    return global_best, global_best_fitness




In [ ]:
# Define  X_train, y_train, X_test, y_test

def fitness_nn(solution):
    n1, n2, lr = int(solution[0]), int(solution[1]), solution[2]
    model = Sequential([
        Dense(n1, input_dim=X_train.shape[1], activation='relu'),
        Dense(n2, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    return mse
